# Введение

# План семинара
Сегодняшний план такой:

* Предсказание ухода клиентов для увеличения retention rate.
* Знакомство с sklearn и быстрое применение метода k ближайших соседей.
* Анализ входных данных, плюсы и минусы k ближайших соседей.
* Анализ более хорошей модели

# Бизнес-задача

## Retention rate
Одной из самых важных бизнес-метрик для многих компаний является customer retention. В разных областях конкретный метод подсчета может отличаться, но это всегда число, которое показывает насколько хорошо мы удерживаем клиента. 

Чем выше customer retention, тем эффективнее тратится бюджет, заложенный на рекламу, потому что уже привлеченный клиент остается с нами дольше. Самый верный способ удерживать клиентов -- это просто создание самого лучшего продукта на рынке. Эта задача очень сложная, но иногда и хорошего продукта не хватает, чтобы клиент не ушел. 

## Подкупаем уходящих клиентов
Чтобы удержать клиента существует чит-код, которым часто пользуются компании: в момент, когда клиент собирается уйти, компания может предложить ему *большую скидку* или как-то еще *улучшить условия*, чтобы удержать клиента. При этом удержав клиента сейчас и потратив какие-то ресурсы, в будущем компания рассчитывает покрыть эти затраты и выйти в плюс. 

Вы наверняка сталкивались с такими предложениями. Вот несколько примеров из жизни:

* Если написать заявление на перенесение номера телефона к другому оператору сотовой связи, то оператор почти всегда предложит вам очень выгодный тариф, не доступный для обычных клиентов.
* Если какое-то время не заказывать доставку еды в Elementary, то они пришлют промокод на 30% скидку.

<a href="https://ibb.co/tXzBbk6"><img src="https://i.ibb.co/8cbMjTw/2021-12-04-13-12-48.png" alt="2021-12-04-13-12-48" border="0"></a>

* Если попробовать закрыть кредитную карту, то вам могут предложить вместо закрытия обнулить плату ежегодную плату за обслуживание.

## Машинное обучение, чтобы находить уходящих клиентов

Если мы подкупим клиента еще до того, как он принял решение уйти, то скорее всего получится уговорить его остаться за меньшее число ресурсов. Чтобы понимать какие клиенты скоро задумаются об уходе, нам нужно делать предсказания. Именно этим и занимается машинное обучение! Сeгодня мы поработаем с открытым датасетом от телеком компании (https://www.kaggle.com/barun2104/telecom-churn). Это табличка с числами, описывающими клиента в какой-то момент + **ушел ли клиент в течение n месяцев с момента описания** (среднее значение этой колонки и есть retention).
<a href="https://ibb.co/swLhH2G"><img src="https://i.ibb.co/zRK1JrY/2021-12-04-12-06-43.png" alt="2021-12-04-12-06-43" border="0"></a>

# Какие инструменты мы будем использовать

Библиотеки питона, которые мы сегодня сипользуем

* pandas - вы уже с ней знакомы, это стандартная библиотека для рабоыт с табличными данными
* sklearn - библиотека с метриками, методами предобработки данных, моделями машинного обучения и многим другим.
* matplotlib - самая популярная библиотека для рисования графиков на питоне.
* seaborn - библиотека для рисования графиков, написанная над matplotlib, имеет дополнительные возможности + если сделать "import seaborn as sns; sns.set()", то она сразу же сделает ваши графики на matplotlib красивее!

# Делаем быстрые предсказания

Чтобы показать всю силу машинного обучения, мы сейчас быстро сделаем предсказания, никак не обрабатывая данные. 

Почти всегда, когда вы хотите решать какую-то реальную задачу, стоит сделать этот этап. Качество, которое мы сейчас получим будет базовым значением, относительно которого мы будем смотреть улучшения от дальшейших действий.

In [ ]:
!unzip archive.zip

Archive:  archive.zip
replace telecom_churn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Прочитаем данные

In [ ]:
import pandas as pd
data = pd.read_csv('telecom_churn.csv')
data

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
...,...,...,...,...,...,...,...,...,...,...,...
3328,0,192,1,1,2.67,2,156.2,77,71.7,10.78,9.9
3329,0,68,1,0,0.34,3,231.1,57,56.4,7.67,9.6
3330,0,28,1,0,0.00,2,180.8,109,56.0,14.44,14.1
3331,0,184,0,0,0.00,2,213.8,105,50.0,7.98,5.0


Разделим табличку на признаки (то, чем можно пользоваться при предсказании) и метки (то, что мы собираемся предсказывать)

In [ ]:
X = data.drop(columns='Churn')
y = data['Churn']

Раздлеим признаки и метки на две части train и validation. Сейчас мы не будем делать test часть, потому что

* Данные для test части стоит собирать отдельно, чтобы она точно не была похожа на train из-за того, что их собрали в одну дату, или собрал один человек, или еще по какой-то нечестной причине.

* У нас мало данных, качество алгоритмов сильно снизится, если мы заберем данные еще и на тест.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

Создадим объект, который делает **класификацию** по **методу ближайших соседей**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn

KNeighborsClassifier()

Вызовем метод .fit(X, y). Для моделей в sklearn этот метод вызывает обучение на предоставленных данных. 
* X - матрица c размерностью [**количество обучающих объектов** x **количество признаков для каждого объекта**], в каждой клетке лежит значение конкрентного признака для конкретного объекта

* y - вектор с размерностью [**количество обучающих объектов**], в каждой клетке лежит 0 или 1. 0 - клиент остался с нами, 1 - клиент ушел.

Метод к ближайших соседей на самом деле не учится, поэтмоу метод .fit(X, y) выполнится очень быстро -- объекту knn нужно просто запомнить данные, которые мы ему дали.

In [ ]:
X_train

,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
2967,149,1,1,2.75,0,147.8,132,76.5,13.84,10.2
660,109,1,0,0.00,1,264.7,69,71.0,15.25,9.5
3069,148,1,1,2.67,1,158.7,91,67.7,8.03,9.9
2617,64,1,0,0.00,2,174.5,98,45.0,9.01,10.7
1161,40,0,0,0.00,3,170.7,55,45.0,8.96,8.2
...,...,...,...,...,...,...,...,...,...,...
1616,67,1,0,0.00,0,179.8,125,46.0,8.66,10.9
1271,81,0,1,2.13,1,237.1,76,84.3,13.21,7.9
1948,128,1,0,0.00,1,148.5,105,46.0,12.15,6.8
1459,95,1,0,0.00,0,197.0,88,50.0,9.52,16.1


In [ ]:
y_train

2967    0
660     1
3069    0
2617    0
1161    0
       ..
1616    0
1271    0
1948    0
1459    0
783     0
Name: Churn, Length: 2499, dtype: int64

In [ ]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

Теперь сделаем предсказания с помощью метода **.predict(X)**. Заметьте, что в метод **.fit(X, y)** мы подавали матрицу с признаками и вектор с правильными ответами. В метод **.predict(X)** мы подаем только матрицу с признаками, потому что мы хотим сделать предсказание и предполагаем, что ответы нам могут быть недоступны. 

Номера колонок в матрице, которую мы подавали в **.fit(X, y)** и в .predict(X) должны совпадать. Т.е. если признак "среднее число минут на звонок" был третьим в матрице, которую мы подали в метод **.fit(X,y)**, то он же должен быть третьим в матрице, которую мы подаем в метод **.predict(X)**

In [ ]:
preds_valid = knn.predict(X_valid)
preds_valid

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

После получения предсказаний нужно проверить, насколько они получились хорошими. Воспользуемся метрикой accuracy

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
print(f'Accuracy={accuracy_score(y_valid, preds_valid)}')

Accuracy=0.854916067146283



Приходим к очень интересной ситуации, accuracy=80%. Т.е. для 80% мы предсказываем  правильный исход. **Кажется это очень хорошо! Идем за шампанским и закрываем вкладку с курсом?🍾🍾🍾🍾?**

* Серди тех, для кого мы предсказали 1 (т.е. решили, что они уйдут), всего лишь половина действительно собиралась уйти
* Среди тех, кто действительно ушел, мы нашли только 27%. Т.е. если бы мы даже предложили максимально щедрое предложение, призванное удержать клиента, мы бы смогли удержать только 27%. 

## Все предсказание в сжатом виде

Ладно, что со такими метриками делать мы еще подумаем, а сейчас соберем все предсказание в одну клетку и понять, что мы написали очень мало кода, который все равно смог получить приемлемые метрики!

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Данные
data = pd.read_csv('telecom_churn.csv')
X = data.drop(columns='Churn')
y = data['Churn']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

# Модель
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
preds_valid = knn.predict(X_valid)

# Тестирование
print(f'Accuracy={accuracy_score(y_valid, preds_valid)}')
print(f'Precision={precision_score(y_valid, preds_valid)}')
print(f'Recall={recall_score(y_valid, preds_valid)}')

Accuracy=0.8717026378896883
Precision=0.5961538461538461
Recall=0.26495726495726496


# Улучшение предсказаний.

## Анализ данных

Когда вы считываете таблички с данными всегда нужно проверить несколько вещей.

<a href="https://ibb.co/vJHt3Nn"><img src="https://i.ibb.co/K9X12tv/2021-12-04-14-09-42.png" alt="2021-12-04-14-09-42" border="0"></a>

In [ ]:
import pandas as pd
data = pd.read_csv('telecom_churn.csv')
data

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
...,...,...,...,...,...,...,...,...,...,...,...
3328,0,192,1,1,2.67,2,156.2,77,71.7,10.78,9.9
3329,0,68,1,0,0.34,3,231.1,57,56.4,7.67,9.6
3330,0,28,1,0,0.00,2,180.8,109,56.0,14.44,14.1
3331,0,184,0,0,0.00,2,213.8,105,50.0,7.98,5.0


### Анализ целевой переменной

Целевая переменная в данном случае Churn. Он = 1, когда клиент ушел, и = 0, когда клиент остался.

Посмотрим на баланс классов

In [ ]:
data['Churn']

0       0
1       0
2       0
3       0
4       0
       ..
3328    0
3329    0
3330    0
3331    0
3332    0
Name: Churn, Length: 3333, dtype: int64

In [ ]:
data['Churn'].mean()

0.14491449144914492

### Теперь посмотрим на признаки. 

Во-первых проверим наличие пропущенных значений. Если пропущенные значения на что-то заменили и мы не увидим их сейчас, то увидим позже, когда нарисуем графики для всех признаков.

In [ ]:
X = data.drop(columns='Churn')

# Посмотрим на разные статистики
X.describe()

,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,0.903090,0.276628,0.816475,1.562856,179.775098,100.435644,56.305161,10.051488,10.237294
std,39.822106,0.295879,0.447398,1.272668,1.315491,54.467389,20.069084,16.426032,2.535712,2.791840
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,0.000000,0.000000
25%,74.000000,1.000000,0.000000,0.000000,1.000000,143.700000,87.000000,45.000000,8.330000,8.500000
50%,101.000000,1.000000,0.000000,0.000000,1.000000,179.400000,101.000000,53.500000,10.070000,10.300000
75%,127.000000,1.000000,1.000000,1.780000,2.000000,216.400000,114.000000,66.200000,11.770000,12.100000
max,243.000000,1.000000,1.000000,5.400000,9.000000,350.800000,165.000000,111.300000,18.190000,20.000000


In [ ]:
is_missing = X.isna()
is_missing

,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
3328,False,False,False,False,False,False,False,False,False,False
3329,False,False,False,False,False,False,False,False,False,False
3330,False,False,False,False,False,False,False,False,False,False
3331,False,False,False,False,False,False,False,False,False,False


In [ ]:
is_missing.sum()

AccountWeeks       0
ContractRenewal    0
DataPlan           0
DataUsage          0
CustServCalls      0
DayMins            0
DayCalls           0
MonthlyCharge      0
OverageFee         0
RoamMins           0
dtype: int64

Ура! Просто пропущенных признаков нет. 

### Посмотрим на типы признаков

**Числовые признаки:**

1. AccountWeeks
2. DataUsage
3. CustServCalls
4. DayMins
5. DayCalls
6. MonthlyCharge
7. OverageFee
8. RoamMins

**Категориальные признаки:**

1. ContractRenewal
2. DataPlan



### Что делать с категориальными признаками 
Очень многие алогритмы на вход требуют вектора из обычных чисел. Если у нас есть категориальный признак, то он обычно кодируется с помощью целых чисел (0, 1, 2, итд) или с помощью строчек. 

Рассмотрим пример. У нас будут данные про каких-то людей. В данный момент не важно, для чего именно мы собираемся использовать эти данные.

In [ ]:
people_data = pd.DataFrame({
    'City': ['Дубай', 'Москва', 'Амстердам', 'Москва', 'Москва'],
    'Weight': [88, 110, -10, 56, 43],
    'Name': ['Андрей', 'Анатолий', 'Unknown', 'Светлана', 'Виктория']
})
people_data

,City,Weight,Name
0,Дубай,88,Андрей
1,Москва,110,Анатолий
2,Амстердам,-10,Unknown
3,Москва,56,Светлана
4,Москва,43,Виктория


Эту табличку **нельзя** подать в почти любую модель машинного обучения, потому что в ней есть строчки. От этих строчек надо как-то избавиться. Первое, что приходит в голову -- заменить строчки на числа. Мы создадим списки из уникальных значений колонки City и Name, заменим каждую строчку на ее индекс в этом списке.

In [ ]:
city_list = list(people_data['City'].unique())
name_list = list(people_data['Name'].unique())
city_list, name_list

(['Дубай', 'Москва', 'Амстердам'],
 ['Андрей', 'Анатолий', 'Unknown', 'Светлана', 'Виктория'])

In [ ]:
people_data['City'] = people_data['City'].apply(lambda city_string: city_list.index(city_string))
people_data['Name'] = people_data['Name'].apply(lambda name_string: name_list.index(name_string))
people_data

,City,Weight,Name
0,0,88,0
1,1,110,1
2,2,-10,2
3,1,56,3
4,1,43,4


Ура, у нас теперь все значения в табличке это числа. Но такие данные все равно нельзя подавать в большинство моделей машинного обучения (про исключения поговорим на соответствующей лекции). 

Для нашей модели города, закодированные целыми числами будут выглядеть как на рисунке ниже. Если мы сразу подадим эти числа в модель, то она не будет знать, что это города, и изначально будет считать, что **Дубай** в каком-то смысле ближе к **Москве**, чем к **Амстердаму**. Т.е. мы навязываем нашей модели порядок городов. Для моделей, основанных на решающих деревьях (слудующее занятия), создание такого порядка может не повредить, но для остальных моделей это плохо.

<a href="https://ibb.co/8m2dwqC"><img src="https://i.ibb.co/Jc2yTV1/2021-12-04-16-31-10.png" alt="2021-12-04-16-31-10" border="0" width=700></a>

Какой же у нас тогда выход? Мы можем закодировать категориальные признаки разными способами, но самый простой и широко применимый это one-hot-encoding. 

<a href="https://ibb.co/F5X21Vh"><img src="https://i.ibb.co/hBcbNfg/2021-12-04-16-45-31.png" alt="2021-12-04-16-45-31" border="0" width=300></a>

In [ ]:
people_data_one_hot = pd.get_dummies(people_data, columns=['City', 'Name'])
people_data_one_hot

,Weight,City_0,City_1,City_2,Name_0,Name_1,Name_2,Name_3,Name_4
0,88,1,0,0,1,0,0,0,0
1,110,0,1,0,0,1,0,0,0
2,-10,0,0,1,0,0,1,0,0
3,56,0,1,0,0,0,0,1,0
4,43,0,1,0,0,0,0,0,1


Вернемся к нашим изначальным данным. У нас всего две колонки с категориальными признаками и при этом значения в них это 0 или 1. Для таких признаков нет смысла кодировать их с помощью one-hot-encoding, оставим их как есть. Просто посмотрим на распределения, чтобы убедиться, что нет аномалий.

In [ ]:
data['ContractRenewal'].value_counts()

1    3010
0     323
Name: ContractRenewal, dtype: int64

In [ ]:
data['DataPlan'].value_counts()

0    2411
1     922
Name: DataPlan, dtype: int64

## Анализ алгоритма

Сегодня мы концентрируемся на k ближайших соседей. Про алгоритм мы уже говорили на лекции.


<a href="https://ibb.co/d6MCxhX"><img src="https://i.ibb.co/QnQSt5h/2021-12-04-22-09-15.png" alt="2021-12-04-22-09-15" border="0"></a>


k ближайших соседей - очень хорошо изученный алгоритм, он даже теоретически идеален: если у нас есть бесконечный датасет для обучения, то с помщоью knn мы можем восстановить полностью всю зависимость. На самом деле, бесконечный датасет даже и не нужен. Нам нужно просто иметь около каждой точкт пространства достатоно много примеров, чтобы мы хорошо могли оценить вероятность каждого из классов.

Самая большая проблема knn - проклятие размерности. Когда у нас становится много признаков, плотность обучающих примеров в n-мерном пространстве резко снижается. 

<a href="https://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/"><img src="https://www.visiondummy.com/wp-content/uploads/2014/04/curseofdimensionality.png" alt="2021-12-04-22-09-15" border="0"></a>

С увеличением размерности, все больше объектов обучающей выборки находятся в "углах" пространства признаков.

<a href="https://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/"><img src="https://www.visiondummy.com/wp-content/uploads/2014/04/sparseness.png" alt="2021-12-04-22-09-15" border="0"></a>


Отсюда вытекают сильные и слабые стороны KNN

Плюсы:
* Хороший бейзлайн.
* Хорошая интерпретируемость при небольом числе признаков.
* Быстрый на небольших и средних данных.
* Теоретически идеален и не делает никаких предположений о зависимости

Минусы:
* Нужно хорошее пространство и метрика, чтобы он работал. Т.е. нужно отбирать признаки, уменьшать размерность данных, стандартизировать признаки итд.
* Если можно сделать какие-то предположения о природе данных, их нужно встраивать в модель через создание новых признаков или измение метрики. Очень сложно и требует опыта.



### Что мы будем делать 

У нас нет пропусков, что значительно облегчает задачу.

* Стандартизируем признаки
* Переберем метрики и k neighbors.

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Данные
data = pd.read_csv('telecom_churn.csv')
X = data.drop(columns='Churn')
y = data['Churn']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)

Стандартизируем признаки/

Представим,  что по оси x1 - площадь квартиры в десятках квадратных метров, а по оси x2 - оценка продавцом уровня шума в квартире из-за дороги. Из-за того, что единицы измерения по оси x2 другие, то ось x1 имеет очень мало веса.

<img src="https://i.stack.imgur.com/OCUmI.png" alt="2021-12-04-22-09-15" border="0">
<img src="https://i.stack.imgur.com/J5r01.png" alt="2021-12-04-22-09-15" border="0">

Познакомимся с новым типом объектов в sklearn: Transformer. У них есть два основных метода .fit() и .transform(). Эти объекты каким-то образом учатся на обучающей выборке и потом могут преобразовывать данные. В нашем случае, Standard Scaler при обучении запоминает среднее и дисперсию для каждого признака. После он нормирует данные, используя их. 

$$x_{new} = \frac{x-mean}{std}$$

Вообще, нормировать признаки стоит почти всегда. Это очень важно для
* Метрических алогритмов
* Линейных алогритмов (мы еще узнаем, что признаки хорошо не только отнормировать, но и как можно сильнее приблизить их распределение к нормальному). Сюда относятся как всевозможные регрессии, так и PCA, который мы используем ниже.

Нормировка не важна разве что для алгоритмов, основанных на решающих деревьях (следующее занятие).

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
scaler.fit(X_train, y_train)
X_train_normed = scaler.transform(X_train)
X_valid_normed = scaler.transform(X_valid)

### Пожнем плоды и обучим модель

In [ ]:
# Модель
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_normed, y_train)
preds_valid = knn.predict(X_valid_normed)

# Тестирование
print(f'Accuracy={accuracy_score(y_valid, preds_valid)}')

Accuracy=0.9016786570743405
